In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 50*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [ ]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240226232152
0	Validation losses: 0.4511, 29.3504, 29.8014	Best Loss: 29.8014 (0)	Accuracy: 83.18%
1	Validation losses: 0.4105, 11.1242, 11.5347	Best Loss: 11.5347 (0)	Accuracy: 85.96%
2	Validation losses: 0.4441, 2.2930, 2.7371	Best Loss: 2.7371 (0)	Accuracy: 84.24%
3	Validation losses: 0.4976, 0.5075, 1.0051	Best Loss: 1.0051 (0)	Accuracy: 81.96%
4	Validation losses: 0.4589, 0.3195, 0.7784	Best Loss: 0.7784 (0)	Accuracy: 83.48%
5	Validation losses: 0.5327, 0.2908, 0.8235	Best Loss: 0.7784 (1)	Accuracy: 79.60%
6	Validation losses: 0.4325, 0.2786, 0.7111	Best Loss: 0.7111 (0)	Accuracy: 85.14%
7	Validation losses: 0.5423, 0.2570, 0.7993	Best Loss: 0.7111 (1)	Accuracy: 80.12%
8	Validation losses: 0.5036, 0.2697, 0.7733	Best Loss: 0.7111 (2)	Accuracy: 82.64%
9	Validation losses: 0.5398, 0.2336, 0.7734	Best Loss: 0.7111 (3)	Accuracy: 80.30%
10	Validation losses: 0.6022, 0.2033, 0.8056	Best Loss: 0.7111 (4)	Accuracy: 80.08%
11	Validation losses: 0

30	Validation losses: 2.3037, 3.1140, 5.4177	Best Loss: 0.6481 (19)	Accuracy: 10.08%
31	Validation losses: 2.3028, 3.1065, 5.4092	Best Loss: 0.6481 (20)	Accuracy: 11.12%
Early stopping!
Total running time:	 2589
INFO:tensorflow:Restoring parameters from ./models/model_20240227004438.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    19.0900003910%
RANDOM SEED:   1 	LAMBDA_EXP:   -4.2
Now: 20240227012749
0	Validation losses: 0.4289, 21.5323, 21.9612	Best Loss: 21.9612 (0)	Accuracy: 84.38%
1	Validation losses: 0.3813, 12.0156, 12.3969	Best Loss: 12.3969 (0)	Accuracy: 86.88%
2	Validation losses: 0.3998, 4.5218, 4.9217	Best Loss: 4.9217 (0)	Accuracy: 85.28%
3	Validation losses: 0.4563, 1.2352, 1.6915	Best Loss: 1.6915 (0)	Accuracy: 83.58%
4	Validation losses: 0.4185, 0.4053, 0.8238	Best Loss: 0.8238 (0)	Accuracy: 84.28%
5	Validation losses: 0.5158, 0.2679, 0.7837	Best Loss: 0.7837 (0)	Accurac

13	Validation losses: 0.5134, 0.1509, 0.6642	Best Loss: 0.6505 (1)	Accuracy: 82.44%
14	Validation losses: 0.4806, 0.1369, 0.6175	Best Loss: 0.6175 (0)	Accuracy: 83.28%
15	Validation losses: 0.5137, 0.1285, 0.6422	Best Loss: 0.6175 (1)	Accuracy: 83.64%
16	Validation losses: 0.5354, 0.1237, 0.6591	Best Loss: 0.6175 (2)	Accuracy: 81.10%
17	Validation losses: 0.5693, 0.1227, 0.6920	Best Loss: 0.6175 (3)	Accuracy: 78.82%
18	Validation losses: 2.3039, 0.5193, 2.8232	Best Loss: 0.6175 (4)	Accuracy: 9.86%
19	Validation losses: nan, nan, nan	Best Loss: 0.6175 (5)	Accuracy: 0.00%
20	Validation losses: nan, nan, nan	Best Loss: 0.6175 (6)	Accuracy: 0.00%
21	Validation losses: nan, nan, nan	Best Loss: 0.6175 (7)	Accuracy: 0.00%
22	Validation losses: nan, nan, nan	Best Loss: 0.6175 (8)	Accuracy: 0.00%
23	Validation losses: nan, nan, nan	Best Loss: 0.6175 (9)	Accuracy: 0.00%
24	Validation losses: nan, nan, nan	Best Loss: 0.6175 (10)	Accuracy: 0.00%
25	Validation losses: nan, nan, nan	Best Loss: 0.617

8	Validation losses: 0.3867, 0.2031, 0.5898	Best Loss: 0.5898 (0)	Accuracy: 85.90%
9	Validation losses: 0.4394, 0.1951, 0.6345	Best Loss: 0.5898 (1)	Accuracy: 84.86%
10	Validation losses: 0.4519, 0.1912, 0.6431	Best Loss: 0.5898 (2)	Accuracy: 83.24%
11	Validation losses: 0.5117, 0.1809, 0.6926	Best Loss: 0.5898 (3)	Accuracy: 80.50%
12	Validation losses: 0.4918, 0.1692, 0.6610	Best Loss: 0.5898 (4)	Accuracy: 81.74%
13	Validation losses: 0.5338, 0.1561, 0.6899	Best Loss: 0.5898 (5)	Accuracy: 82.44%
14	Validation losses: 0.4799, 0.1360, 0.6159	Best Loss: 0.5898 (6)	Accuracy: 82.88%
15	Validation losses: 0.4456, 0.1285, 0.5742	Best Loss: 0.5742 (0)	Accuracy: 84.88%
16	Validation losses: 0.4700, 0.1154, 0.5855	Best Loss: 0.5742 (1)	Accuracy: 83.82%
17	Validation losses: 0.4228, 0.1050, 0.5277	Best Loss: 0.5277 (0)	Accuracy: 84.98%
18	Validation losses: 0.4379, 0.1057, 0.5436	Best Loss: 0.5277 (1)	Accuracy: 84.64%
19	Validation losses: 0.4852, 0.1052, 0.5904	Best Loss: 0.5277 (2)	Accuracy: 8

40	Validation losses: 0.3532, 0.0838, 0.4370	Best Loss: 0.4370 (0)	Accuracy: 87.56%
41	Validation losses: 0.3553, 0.0838, 0.4391	Best Loss: 0.4370 (1)	Accuracy: 87.58%
42	Validation losses: 0.3554, 0.0837, 0.4391	Best Loss: 0.4370 (2)	Accuracy: 87.68%
43	Validation losses: 0.3596, 0.0839, 0.4436	Best Loss: 0.4370 (3)	Accuracy: 87.28%
44	Validation losses: 0.3611, 0.0842, 0.4453	Best Loss: 0.4370 (4)	Accuracy: 87.26%
45	Validation losses: 0.3633, 0.0843, 0.4476	Best Loss: 0.4370 (5)	Accuracy: 87.08%
46	Validation losses: 0.3694, 0.0840, 0.4534	Best Loss: 0.4370 (6)	Accuracy: 86.96%
47	Validation losses: 0.3699, 0.0845, 0.4544	Best Loss: 0.4370 (7)	Accuracy: 86.92%
48	Validation losses: 0.3896, 0.0850, 0.4746	Best Loss: 0.4370 (8)	Accuracy: 86.06%
49	Validation losses: 0.3778, 0.0849, 0.4627	Best Loss: 0.4370 (9)	Accuracy: 87.18%
50	Validation losses: 0.3692, 0.0851, 0.4543	Best Loss: 0.4370 (10)	Accuracy: 87.10%
51	Validation losses: 0.3710, 0.0849, 0.4559	Best Loss: 0.4370 (11)	Accurac

...

17	Validation losses: 0.3642, 0.1606, 0.5247	Best Loss: 0.5220 (2)	Accuracy: 87.68%
18	Validation losses: 0.3537, 0.1564, 0.5101	Best Loss: 0.5101 (0)	Accuracy: 87.76%
19	Validation losses: 0.3500, 0.1561, 0.5061	Best Loss: 0.5061 (0)	Accuracy: 87.50%
20	Validation losses: 0.3332, 0.1541, 0.4872	Best Loss: 0.4872 (0)	Accuracy: 88.34%
21	Validation losses: 0.3652, 0.1459, 0.5111	Best Loss: 0.4872 (1)	Accuracy: 87.10%
22	Validation losses: 0.3984, 0.1384, 0.5369	Best Loss: 0.4872 (2)	Accuracy: 85.86%
23	Validation losses: 0.3447, 0.1328, 0.4775	Best Loss: 0.4775 (0)	Accuracy: 87.76%
24	Validation losses: 0.3331, 0.1280, 0.4611	Best Loss: 0.4611 (0)	Accuracy: 88.04%
25	Validation losses: 0.3158, 0.1211, 0.4370	Best Loss: 0.4370 (0)	Accuracy: 88.82%
26	Validation losses: 0.3329, 0.1170, 0.4499	Best Loss: 0.4370 (1)	Accuracy: 88.42%
27	Validation losses: 0.3036, 0.1139, 0.4175	Best Loss: 0.4175 (0)	Accuracy: 88.78%
28	Validation losses: 0.3312, 0.1103, 0.4415	Best Loss: 0.4175 (1)	Accuracy:

52	Validation losses: 0.2997, 0.1024, 0.4021	Best Loss: 0.3852 (13)	Accuracy: 89.06%
53	Validation losses: 0.3249, 0.1039, 0.4288	Best Loss: 0.3852 (14)	Accuracy: 88.92%
54	Validation losses: 0.3208, 0.1046, 0.4254	Best Loss: 0.3852 (15)	Accuracy: 88.90%
55	Validation losses: 0.3309, 0.1051, 0.4360	Best Loss: 0.3852 (16)	Accuracy: 87.88%
56	Validation losses: 0.3255, 0.1063, 0.4318	Best Loss: 0.3852 (17)	Accuracy: 88.44%
57	Validation losses: 0.3273, 0.1071, 0.4345	Best Loss: 0.3852 (18)	Accuracy: 88.42%
58	Validation losses: 0.3246, 0.1068, 0.4314	Best Loss: 0.3852 (19)	Accuracy: 88.68%
59	Validation losses: 0.3396, 0.1076, 0.4472	Best Loss: 0.3852 (20)	Accuracy: 88.00%
Early stopping!
Total running time:	 4913
INFO:tensorflow:Restoring parameters from ./models/model_20240227231213.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    10.2200007439%
RANDOM SEED:   1 	LAMBDA_EXP:   -5.4
Now:

45	Validation losses: 0.3186, 0.0981, 0.4167	Best Loss: 0.3900 (7)	Accuracy: 90.34%
46	Validation losses: 0.3271, 0.0990, 0.4261	Best Loss: 0.3900 (8)	Accuracy: 89.92%
47	Validation losses: 0.3340, 0.1003, 0.4342	Best Loss: 0.3900 (9)	Accuracy: 89.46%
48	Validation losses: 0.3143, 0.1015, 0.4158	Best Loss: 0.3900 (10)	Accuracy: 90.26%
49	Validation losses: 0.3560, 0.1030, 0.4590	Best Loss: 0.3900 (11)	Accuracy: 89.54%
50	Validation losses: 0.3122, 0.1046, 0.4169	Best Loss: 0.3900 (12)	Accuracy: 89.60%
51	Validation losses: 0.3223, 0.1069, 0.4291	Best Loss: 0.3900 (13)	Accuracy: 89.24%
52	Validation losses: 0.3160, 0.1090, 0.4251	Best Loss: 0.3900 (14)	Accuracy: 89.38%
53	Validation losses: 0.3033, 0.1110, 0.4143	Best Loss: 0.3900 (15)	Accuracy: 89.84%
54	Validation losses: 0.3066, 0.1135, 0.4200	Best Loss: 0.3900 (16)	Accuracy: 89.34%
55	Validation losses: 0.3247, 0.1153, 0.4400	Best Loss: 0.3900 (17)	Accuracy: 89.10%
56	Validation losses: 0.3159, 0.1166, 0.4324	Best Loss: 0.3900 (18)	

19	Validation losses: 0.3701, 0.2133, 0.5834	Best Loss: 0.5382 (1)	Accuracy: 87.50%
20	Validation losses: 0.3314, 0.2064, 0.5378	Best Loss: 0.5378 (0)	Accuracy: 88.02%
21	Validation losses: 0.3309, 0.1923, 0.5232	Best Loss: 0.5232 (0)	Accuracy: 88.24%
22	Validation losses: 0.3043, 0.1804, 0.4846	Best Loss: 0.4846 (0)	Accuracy: 88.92%
23	Validation losses: 0.3013, 0.1700, 0.4713	Best Loss: 0.4713 (0)	Accuracy: 89.42%
24	Validation losses: 0.3088, 0.1622, 0.4711	Best Loss: 0.4711 (0)	Accuracy: 89.00%
25	Validation losses: 0.3892, 0.1556, 0.5448	Best Loss: 0.4711 (1)	Accuracy: 86.28%
26	Validation losses: 0.2939, 0.1492, 0.4431	Best Loss: 0.4431 (0)	Accuracy: 89.90%
27	Validation losses: 0.3082, 0.1448, 0.4530	Best Loss: 0.4431 (1)	Accuracy: 89.32%
28	Validation losses: 0.3094, 0.1401, 0.4495	Best Loss: 0.4431 (2)	Accuracy: 89.40%
29	Validation losses: 0.2980, 0.1361, 0.4341	Best Loss: 0.4341 (0)	Accuracy: 89.60%
30	Validation losses: 0.2845, 0.1324, 0.4169	Best Loss: 0.4169 (0)	Accuracy:

1	Validation losses: 0.3599, 1.0657, 1.4257	Best Loss: 1.4257 (0)	Accuracy: 87.18%
2	Validation losses: 0.3492, 1.0355, 1.3846	Best Loss: 1.3846 (0)	Accuracy: 87.30%
3	Validation losses: 0.3783, 0.9958, 1.3740	Best Loss: 1.3740 (0)	Accuracy: 86.38%
4	Validation losses: 0.3300, 0.9478, 1.2777	Best Loss: 1.2777 (0)	Accuracy: 87.10%
5	Validation losses: 0.3282, 0.8930, 1.2212	Best Loss: 1.2212 (0)	Accuracy: 87.50%
6	Validation losses: 0.3306, 0.8325, 1.1631	Best Loss: 1.1631 (0)	Accuracy: 87.60%
7	Validation losses: 0.3687, 0.7686, 1.1373	Best Loss: 1.1373 (0)	Accuracy: 86.30%
8	Validation losses: 0.2869, 0.7025, 0.9895	Best Loss: 0.9895 (0)	Accuracy: 89.60%
9	Validation losses: 0.3475, 0.6369, 0.9844	Best Loss: 0.9844 (0)	Accuracy: 87.30%
10	Validation losses: 0.3592, 0.5720, 0.9312	Best Loss: 0.9312 (0)	Accuracy: 86.82%
11	Validation losses: 0.3901, 0.5106, 0.9007	Best Loss: 0.9007 (0)	Accuracy: 83.52%
12	Validation losses: 0.3316, 0.4527, 0.7843	Best Loss: 0.7843 (0)	Accuracy: 87.78%
1